In [1]:
!pip install -U spacy==2.3.5

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

In [4]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [5]:
def is_int(n):
    try:
        float_n = float(n)
        int_n = int(float_n)
    except ValueError:
        return False
    else:
        return float_n == int_n

# test
print(is_int('test'))
print(is_int('4'))
print(is_int('4.5'))

False
True
False


In [6]:
text = "(3) Uavhengig av fartøyets byggedato skal gassfyrte anlegg skal tilfredsstille forskrift 20. oktober 1983 nr. 1580 om sikringstiltak for gassfyrte anlegg m.m. som bruker propan eller andre lette kullvannstoffer anvendt ombord på fartøy. (1) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 1992 med elektriske anlegg med spenning inntil 50 V skal oppfylle kravene i Nordisk Båt Standard (1990) Y11 om elektriske anlegg."

In [7]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher

# Blank model.
# Does not have a POS tagger.
nlp = English()

doc = nlp(text)

matcher = Matcher(nlp.vocab)

#
# START - spaCy patterns
#

# ALTERNATIVE_REFERENCE
alternative_reference_patterns = [
    [
        {"LOWER": {"IN": ["nr"]}}, 
        {"IS_PUNCT": True}, 
        {'IS_DIGIT': True}],
    [
        {"LOWER": {"IN": ["nordisk"]}},
        {"LOWER": {"IN": ["båt"]}},
        {"LOWER": {"IN": ["standard"]}},
        {"LOWER": {"IN": ["("]}},
        {'IS_DIGIT': True},
        {"LOWER": {"IN": [")"]}}
    ]
]

matcher.add("ALTERNATIVE_REFERENCE", alternative_reference_patterns)

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]
    final_token_start = token_start
    final_token_end = token_end

    spacy_pattern_detection = doc[token_start:token_end]
    spacy_pattern_detection_as_lower_text = spacy_pattern_detection.text.lower()
    
    if (match_id_as_string == "ALTERNATIVE_REFERENCE" and
        token_start > 5
        and spacy_pattern_detection_as_lower_text.startswith('nr. ')): # Example: nr. 1580
        start_word = "forskrift"
        if doc[token_start - 1].text.lower() == start_word:
            final_token_start = token_start - 1
        elif doc[token_start - 2].text.lower() == start_word:
            final_token_start = token_start - 2
        elif doc[token_start - 3].text.lower() == start_word:
            final_token_start = token_start - 3
        elif doc[token_start - 4].text.lower() == start_word:
            final_token_start = token_start - 4
        elif doc[token_start - 5].text.lower() == start_word:
            final_token_start = token_start - 5

    #
    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    #
    span = doc[final_token_start:final_token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    # return result
    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"ALTERNATIVE_REFERENCE": "darksalmon"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)

# Functions run from .py file

In [8]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import spacy_matching_rule_identify_alternative_reference_no

from spacy_matching_rule_identify_alternative_reference_no import identify_alternative_reference_in_text

importlib.reload(spacy_matching_rule_identify_alternative_reference_no)

################
################

identify_alternative_reference_in_text(text)

[{'start': 79, 'end': 114, 'label': 'ALTERNATIVE_REFERENCE'},
 {'start': 381, 'end': 408, 'label': 'ALTERNATIVE_REFERENCE'}]